In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATASETS
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE CELL.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote
from urllib.error import HTTPError
from zipfile import ZipFile

CHUNK_SIZE = 40960 
DATASET_MAPPING = 'jane-street-market-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F23304%2F1781886%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20201216%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20201216T221901Z%26X-Goog-Expires%3D259199%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D01ef84d3797bfae58b23ef6888ef2068f32c0ba8107b371d74c9926b52ebdfbfe83c67ebd7380a28c3df5522549d635eb6431c19b062941fd3a754b64b3d07254bda8a8f124309e06baadd86c2e6757b9d6091c69cc62afc53f7eac271c5d09103a87b73706914b1c6840f84e9bc8a913f57f7887d77ea3df89bc2bed934aea19d7c552519ab9df7b6e755732a975baa069b98441463aa4456b4c8d02b32187aa1dc7903056036accbaaf4e5f2c88fb8ea71c894465155ccab0f59262d7f6b3ae54940c9541532c82b3e7b11265e7fde87fee2da9fea76eb0d1e12f7af0c14bf0545151f1a3a3623dcd4c68bdc091772e55b0b1ed7fe8c3cfee6733cc1ff8d2c'
KAGGLE_INPUT_PATH='/home/kaggle/input'
KAGGLE_INPUT_SYMLINK='/kaggle'

os.makedirs(KAGGLE_INPUT_PATH, 777)
os.symlink(KAGGLE_INPUT_PATH, os.path.join('..', 'input'), target_is_directory=True)
os.makedirs(KAGGLE_INPUT_SYMLINK)
os.symlink(KAGGLE_INPUT_PATH, os.path.join(KAGGLE_INPUT_SYMLINK, 'input'), target_is_directory=True)

for dataset_mapping in DATASET_MAPPING.split(','):
    directory, download_url_encoded = dataset_mapping.split(':')
    download_url = unquote(download_url_encoded)
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as zipfileres, NamedTemporaryFile() as tfile:
            total_length = zipfileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes zipped')
            dl = 0
            data = zipfileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = zipfileres.read(CHUNK_SIZE)
            print(f'\nUnzipping {directory}')
            with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue
print('Dataset import complete.')


# Jane Street: XGBoost HyperOpt + GroupKFold

I've made a notebook for hyperparameter tuning for XGBoost/LightGBM/CatBoost models.

I have also added a decision threshold optimisation method based on different index methods (e.g., Youden Index) for ROC curve.

I use the GroupTimeSplitKFold method. 

Modified from these excellent notebooks:

https://www.kaggle.com/hamditarek/market-prediction-xgboost-with-gpu-fit-in-1min

https://www.kaggle.com/xhlulu/jane-street-cudf-xgboost-with-gpu

https://www.kaggle.com/jorijnsmit/found-the-holy-grail-grouptimeseriessplit

In [ ]:
import os, gc
import cudf
import numpy as np
import cupy as cp
import pandas as pd
import janestreet
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Preprocessing

In [ ]:
print('Loading...')
train = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
features = [c for c in train.columns if 'feature' in c]

print('Filling...')
# f_mean = train[features[1:]].mean()
train = train.query('weight > 0').reset_index(drop = True)
# train[features[1:]] = train[features[1:]].fillna(f_mean)
train['action'] = (train['resp'] > 0).astype('int')

# print('Converting...')
# train = train.to_pandas()
# f_mean = f_mean.values.get()
# np.save('f_mean.npy', f_mean)

print('Finish.')

# HyperOpt

In [ ]:
# def optimise(params):
    
#     print(params)
#     p = {'learning_rate': params['learning_rate'],
#          'max_depth': params['max_depth'], 
#          'gamma': params['gamma'], 
#          'min_child_weight': params['min_child_weight'], 
#          'subsample': params['subsample'], 
#          'colsample_bytree': params['colsample_bytree'], 
#          'verbosity': 0, 
#          'objective': 'binary:logistic',
#          'eval_metric': 'auc', 
#          'tree_method': 'gpu_hist', 
#          'random_state': 42,
#         }
    
#     oof = np.zeros(len(train['action']))
#     gkf = GroupKFold(n_splits = 5)
#     for fold, (tr, te) in enumerate(gkf.split(train['action'].values.get(), train['action'].values.get(), train['date'].values.get())):
#         X_tr, X_val = train.loc[tr, features], train.loc[te, features]
#         y_tr, y_val = train.loc[tr, 'action'], train.loc[te, 'action']

#         d_tr = xgb.DMatrix(X_tr, y_tr)
#         d_val = xgb.DMatrix(X_val, y_val)
#         clf = xgb.train(p, d_tr, 10000, [(d_val, 'eval')], early_stopping_rounds = 100, verbose_eval = 0)
#         oof[te] += clf.predict(d_val, ntree_limit = clf.best_ntree_limit)
# #         score = roc_auc_score(y_val.to_array(), oof[te])
# #         print(f'Fold {fold} ROC AUC:\t', score)
#         rubbish = gc.collect()
        
#     score_oof = roc_auc_score(train['action'].values.get(), oof)
#     print(score_oof)
#     return - score_oof

In [ ]:
# param_space = {'learning_rate': hp.uniform('learning_rate', 0.01, 0.3), 
#                'max_depth': scope.int(hp.quniform('max_depth', 3, 11, 1)), 
#                'gamma': hp.uniform('gamma', 0, 10), 
#                'min_child_weight': hp.uniform('min_child_weight', 0, 10),
#                'subsample': hp.uniform('subsample', 0.5, 1), 
#                'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1), 
#               }

# trials = Trials()

# hopt = fmin(fn = optimise, 
#             space = param_space, 
#             algo = tpe.suggest, 
#             max_evals = 30, 
#             trials = trials, 
#            )

In [ ]:
# print(hopt)

# GroupTimeSplitKFold

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

# Training

In [ ]:
p_best = {'learning_rate': 0.050055662268729532,
          'max_depth': 6, 
          'gamma': 0.07902741481945934, 
          'min_child_weight': 9.9404564544994, 
          'subsample': 0.7001330243186357, 
          'colsample_bytree': 0.7064645381596891, 
          'objective': 'binary:logistic',
          'eval_metric': 'auc', 
          'tree_method': 'gpu_hist', 
          'random_state': 42,
         }

models = []
val_idx = []
oof = np.zeros(len(train['action']))
gkf = GroupTimeSeriesSplit(n_splits = 5)
# gkf = GroupKFold(n_splits = 5)
for fold, (tr, te) in enumerate(gkf.split(train['action'].values.get(), groups = train['date'].values.get())):
    val_idx.append(te)
    X_tr, X_val = train.loc[tr, features], train.loc[te, features]
    y_tr, y_val = train.loc[tr, 'action'], train.loc[te, 'action']
    
    d_tr = xgb.DMatrix(X_tr, y_tr)
    d_val = xgb.DMatrix(X_val, y_val)
    clf = xgb.train(p_best, d_tr, 10000, [(d_val, 'eval')], early_stopping_rounds = 100, verbose_eval = 0)
    oof[te] += clf.predict(d_val, ntree_limit = clf.best_ntree_limit)
    score = roc_auc_score(y_val.values.get(), oof[te])
    print(f'Fold {fold} ROC AUC:\t', score)
    models.append(clf)
    
    del X_tr, X_val, y_tr, y_val, d_tr, d_val
    rubbish = gc.collect()
    
val_idx = np.concatenate(val_idx)

In [ ]:
score_oof = roc_auc_score(train['action'].values.get()[val_idx], oof[val_idx])
print(score_oof)

# Decision Threshold Optimisation

In [ ]:
fpr, tpr, th = roc_curve(train['action'].values.get()[val_idx], oof[val_idx])
plt.figure(figsize = (8, 6))
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize = 14)
plt.plot(fpr, tpr, 'r--', label = 'AUC = %0.5f' % score_oof, ms = 5, markerfacecolor = 'none', linewidth = '3')
plt.legend(loc = 'lower right', fontsize = 12)
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0.01, 1])
plt.ylim([0.00001, 1])
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.ylabel('True Positive Probability', fontsize = 12)
plt.xlabel('False Positive Probability', fontsize = 12)
plt.show()

In [ ]:
# def acc_thred(th, fpr, tpr, name, index):
#     fpr_ct = cp.array(fpr)
#     tpr_ct = cp.array(tpr)
#     th_ct = cp.array(th)
#     # Choose the decision threshold optimisation index
#     if index == 'Euclidean':
#         # Euclidean Index
#         opt_idx = cp.argmin(cp.sqrt(cp.square(1-tpr_ct) + cp.square(fpr_ct)))
#     elif index == 'Youden':
#         # Youden Index
#         opt_idx = cp.argmax(tpr_ct-fpr_ct)
#     else:
#         # Unknown Index
#         print('Unkown Index. Reset to Youden Index.')
#         opt_idx = cp.argmax(tpr_ct-fpr_ct)
#     opt_fpr = fpr_ct[opt_idx]
#     opt_tpr = tpr_ct[opt_idx]
#     opt_th = th_ct[opt_idx]
#     print('Best False Positive Probability ('+name+'):\t', opt_fpr)
#     print('Best True Positive Probability ('+name+'):\t', opt_tpr)
#     print('Best Decision Threshold ('+name+'):\t\t', opt_th)
#     return opt_fpr.item(), opt_tpr.item(), opt_th.item()

In [ ]:
# opt_fpr, opt_tpr, opt_th = acc_thred(th, fpr, tpr, 'XGB', 'Youden')

# Optimise Threshold By Utility Score

In [ ]:
# def utility_score(df, action):
#     df['actionv'] = action
#     count_i = len(df['date'].unique())
#     Pi = cp.zeros(count_i)
#     for i, day in enumerate(df['date'].unique().values.tolist()):
#         tmp = df.loc[df['date'] == day, ['weight', 'resp', 'actionv']]
#         Pi[i] = (tmp['weight'] * tmp['resp'] * tmp['actionv']).sum()
    
#     t = cp.sum(Pi) / cp.sqrt(cp.sum(Pi ** 2)) * cp.sqrt(250 / count_i)
#     u = cp.minimum(cp.maximum(t, 0), 6) * cp.sum(Pi)
#     del df['actionv']
#     return u.item()

In [ ]:
# oof = cp.array(oof)
# opt_th = 0
# opt_u = 0
# gap = oof.max() - oof.min()
# bins = 50
# for th in tqdm(cp.arange(oof.min(), oof.max(), gap.get() / bins)):
#     action = cp.where(oof >= th, 1, 0)
#     u = utility_score(train, action)
#     if u > opt_u:
#         opt_th = th.item()
#         opt_u = u
#         print(opt_th, opt_u)

# Example Test Prediction Analysis

In [ ]:
# example_test = cudf.read_csv('../input/jane-street-market-prediction/example_test.csv')
# d_tt = xgb.DMatrix(example_test.loc[:, features])
# test_preds = np.zeros(example_test.shape[0])
# for i, clf in enumerate(models):
#     test_preds += clf.predict(d_tt, ntree_limit = clf.best_ntree_limit) / len(models)
# print(test_preds.min())
# print(test_preds.max())
# print(test_preds.mean())
# print(test_preds.std())
# plt.hist(test_preds, bins = 100)
# plt.show()

# Submitting

In [ ]:
env = janestreet.make_env()
env_iter = env.iter_test()

In [ ]:
opt_th = 0.5
for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
#         x_tt = test_df.loc[:, features].values
#         if np.isnan(x_tt[:, 1:].sum()):
#             x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        d_tt = xgb.DMatrix(test_df.loc[:, features])
        for i, clf in enumerate(models):
            if i == 0:
                pred = clf.predict(d_tt, ntree_limit = clf.best_ntree_limit) / len(models)
            else:
                pred += clf.predict(d_tt, ntree_limit = clf.best_ntree_limit) / len(models)
#         pred = models[-1].predict(d_tt, ntree_limit = models[-1].best_ntree_limit)
        pred_df.action = np.where(pred >= opt_th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)